**Step # 01 Install All the Required Packages**

In [ ]:
!pip install ultralytics

**Step # 02 Import All the Required Libraries**

In [ ]:
import ultralytics
from ultralytics import YOLO

**Step # 03 Check Ultralytics Version and Setup Completion**

In [ ]:
ultralytics.checks()

**Step 04: Download the Dataset from Roboflow**

In [ ]:
!pip install roboflow

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="z1uD0b9XOZdnivUdFkEd")
project = rf.workspace("pose-detection-twxbg").project("human-activity-ce7zu")
version = project.version(2)
dataset = version.download("yolov8") # "yolov11" does not work

In [ ]:
!gdown "https://drive.google.com/uc?id=1kSI3LP8hVRw7dY678sH9OGmdnH7Zi-CE&confirm=t"

In [ ]:
!mkdir dataset

In [ ]:
%cd "/content/dataset/"

In [ ]:
!unzip "/content/Human activity.v2i.yolov8.zip"

In [ ]:
!pwd
%cd /content/

In [ ]:
# *** Change path in data.yaml file for Google Colab ***
# path: /content/dataset
# test: test/images    # relative to 'path'
# train: train/images  # relative to 'path'
# val: valid/images    # relative to 'path'

**Step # 05 Load the YOLO11 Model**

In [ ]:
model = YOLO("yolo11s-pose.pt")

**Step # 06 Train the YOLO11 Pose Model**

In [ ]:
! rm -rf /content/runs/pose

In [ ]:
results = model.train(data = "/content/Human-activity-2/data.yaml", task = "pose", mode = "train", imgsz = 640, epochs = 10, batch = 8)

**Step # 07 Validate Custom Model**

In [ ]:
#Validate the Model
metrics = model.val()

In [ ]:
metrics.box.map

In [ ]:
metrics.box.map50

In [ ]:
metrics.box.map75

In [ ]:
metrics.box.maps

**Step # 08 Results Analysis**

**Confusion Matrix**

In [ ]:
from IPython.display import Image, display
Image("/content/runs/pose/train/confusion_matrix.png", width = 800)

In [ ]:
from IPython.display import Image, display
Image("/content/runs/pose/train/confusion_matrix_normalized.png", width = 800)

**Percision Curve**

In [ ]:
Image("/content/runs/pose/train/PoseP_curve.png", width = 800)

**Recall Curve**

In [ ]:
Image("/content/runs/pose/train/PoseR_curve.png", width = 800)

**Model Predictions on Validation Batch**

In [ ]:
Image("/content/runs/pose/train/val_batch1_pred.jpg", width = 800)

In [ ]:
Image("/content/runs/pose/train/val_batch2_pred.jpg", width = 800)

**Step # 09 Inference on Test Dataset Images using the Trained/ Fine-Tune Pose Estimation Model**

In [ ]:
!gdown "https://drive.google.com/uc?id=1H3cfzzH9v3T9k9TNj8oaUy_sf-PH1kTA&confirm=t"

In [ ]:
#Load Trained Model
model = YOLO("/content/best.pt")

#Specify the test folder path
test_path = "/content/dataset/test/images"

#List all the images in the test folder
import os
image_files = [os.path.join(test_path, img) for img in os.listdir(test_path) if img.endswith(('.jpg', '.jpeg', '.png'))]

#Predict each images
for img_path in image_files:
  results = model.predict(source = img_path, save = True, conf = 0.25)

**Step # 10 Plot Prediction on Test Images**

In [ ]:
# Plot prediction on test images
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Specify the predict folder path
pred_path = '/content/runs/pose/predict'

# List all images in the predict folder
pred_image_files = [os.path.join(pred_path, img) for img in os.listdir(pred_path) if img.endswith(('.jpg', '.jpeg', '.png'))]

# Number of images per row
images_per_row = 5

# Calculate the number of rows needed
n_rows = len(pred_image_files) // images_per_row + int(len(pred_image_files) % images_per_row != 0)

# Set figure size
fig, axs = plt.subplots(n_rows, images_per_row, figsize=(15, 3 * n_rows))

# Flatten axes if necessary (for easier iteration)
axs = axs.flatten()

# Loop through each image and display it
for i, img_path in enumerate(pred_image_files):
    img = mpimg.imread(img_path)      # Read the image
    axs[i].imshow(img)                # Show the image
    axs[i].axis('off')                # Turn off axis
    axs[i].set_title(f"Image {i+1}")  # Add title

# Hide any extra empty subplots (if the number of images is not a perfect multiple of images_per_row)
for j in range(i+1, len(axs)):
    axs[j].axis('off')  # Hide unused axes

# Display all images
plt.tight_layout()
plt.show()


**Step # 10 Inference on Video**

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!gdown "https://drive.google.com/uc?id=19jREqMUWoJgixJgwgeHvvsPu82neJITp&confirm=t"

In [ ]:
results = model.predict(source = "/content/video1.mp4", save = True, conf = 0.25, iou = 0.2)

In [ ]:
!rm "/content/result_compressed.mp4"

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os

# Input video path
save_path = '/content/runs/pose/predict/video1.avi'

# Compressed video path
compressed_path = "/content/result_compressed.mp4"

os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)